In [1]:
import numpy as np
import pandas as pd
import glob
import sys,os
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from collections import defaultdict
import pymp
import qgrid
import plotly.express as px
import gezi
from gezi import tqdm, line
# tqdm.pandas()
from IPython.display import display
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

No module named 'wandb.keras'


In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [11]:
v = '6'
mark = 'offline'
root = f'../working/{mark}/{v}'
metrics = [
    'score', 'cls/score', 'reg/score',
    'r@1', 'r@2', 'r@3', 'r@40'
]

In [14]:
def gen_metrics(root, folds=1):
  dfs= []
  for fold in range(folds):
    pattern = f'{root}/{fold}/*/metrics.csv'
    files = glob.glob(pattern)
    if not files:
      continue
    files = sorted(files, key=lambda x: os.path.getmtime(x))
    for file in tqdm(files):
      if not gezi.non_empty(file):
        continue
      df = pd.read_csv(file)
      if not len(df):
        continue
      if 'step' not in df.columns:
        df['step'] = range(len(df))
      model = os.path.basename(os.path.dirname(file))
      df['model'] = model
      df['mtime'] = os.path.getmtime(file)
      df['ctime'] = os.path.getctime(file)
      df['fold'] = fold
      dfs.append(df)
  df = pd.concat(list(dfs))
  return df

In [15]:
df = gen_metrics(root)

  0%|          | 0/220 [00:00<?, ?it/s]

In [17]:
models = set(df.model)
# mark = 'longformer'
# models = [x for x in models if mark in x]
# mark = '.len'
# models = [x for x in models if mark not in x]
df = df[df.model.isin(models)]

In [25]:
line(df[df.model.str.contains('context3.n_context-30')], 'score/reg', x='step', color='model')